In [9]:
import numpy as np
import tqdm
import json
from PIL import Image
import torch
import os
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import pandas as pd
from torch.nn import AdaptiveAvgPool2d
import pickle

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self):
        super(MyIterableDataset).__init__()
        
    def __iter__(self):
        for i in range(len(paths)):
            yield tfms(Image.open(paths[i]))

In [3]:
%%time

DATA_PATH = 'C:\\Users\\A_Pot\\Documents\\WorkProjects\\raw_data\\vdc\\veriwild'

paths = []
idxs = []
idx = 0
for one in os.listdir(DATA_PATH):
    for two in os.listdir(os.path.join(DATA_PATH, one)):
        img_names = os.listdir(os.path.join(DATA_PATH, one, two))
        paths += list(map(lambda x: os.path.join(DATA_PATH, one, two, x), img_names))
        idxs += [idx] * len(img_names) 
        idx += 1

Wall time: 15.9 s


In [4]:
len(paths), len(idxs)

(416314, 416314)

In [6]:
model = EfficientNet.from_pretrained('efficientnet-b7').cuda()

tfms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

loader = torch.utils.data.DataLoader(MyIterableDataset(), num_workers=0, shuffle=False, batch_size=10)

Loaded pretrained weights for efficientnet-b7


In [10]:
%%time
features = []
for batch in tqdm.tqdm(loader):
    features.append(AdaptiveAvgPool2d(output_size=1)(model.extract_features(batch.cuda())).detach().cpu().numpy().squeeze())
features = np.concatenate(features, axis=0)    

41632it [2:48:47,  4.11it/s]

Wall time: 2h 48min 47s


In [10]:
features.shape

(416314, 2560)

In [11]:
with open('efficientnet-b7.pickle', 'wb') as f:
    pickle.dump(features, f)